### Ko GPT2
- https://huggingface.co/skt/kogpt2-base-v2
--> https://github.com/SKT-AI/KoGPT2
- 상기 git 에 있는 내용 복붙

In [2]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

In [11]:
text = '''
유구한 역사와 전통에 빛나는 우리 대한민국은 3.1운동으로 건립된 대한민국임시정부와 법통과 불의에 항거한
'''


import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.eval()

# 프롬프트 엔지니어링
# 프롬프트가 중요함!!!! 한줄요약이 아닌, 핵심단어 등등의 프롬프트에 따라 달라지므로.
prompt = f'{text}\n\n 한줄 요약: '
# 인코딩 (프롬프트를 모델에 넣기 위해서 토크나이징)
input_ids = tokenizer.encode(prompt, return_tensors='pt')
# 모델 추론 (생성)
with torch.no_grad():
    gen_ids = model.generate(input_ids,
                            max_length=256,
                            max_new_tokens = 100,
                            repetition_penalty=2.0, # 같은 말을 반복하지 않도록 벌점을 줌
                            pad_token_id=tokenizer.pad_token_id, # 원래 이 모델이 갖고 있던것? 중요하지않음..
                            eos_token_id=tokenizer.eos_token_id, # 원래 이 모델이 갖고 있던것? 중요하지않음..
                            bos_token_id=tokenizer.bos_token_id, # 원래 이 모델이 갖고 있던것? 중요하지않음..
                            use_cache=True,
                            do_sample=True, #자연스러운 문장을 만들어줌
                            temperature=0.7, #창의성 조정 (낮을수록 보수적, 높을수록 환각 가능성 높음)
                            top_k=50 #확률적으로 상위 50개 단어중에서 선택
                            )
    generated = tokenizer.decode(gen_ids[0])

    # 프롬프트 이후에 생성된 텍스트만 추출
    summary = generated[len(prompt):].strip()
    print(generated)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Both `max_new_tokens` (=100) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



유구한 역사와 전통에 빛나는 우리 대한민국은 3.1운동으로 건립된 대한민국임시정부와 법통과 불의에 항거한 


 한줄 요약: | 1919년 2월 25일 - 1917년 11월 30일 05월 1일 ~ 1915년 3월 26일 11시 기준)
일제강점기에 독립운동의 주체였던 애국지사는 일제의 탄압으로 인해 대부분 반파되었다.
이후 독립유공자와 유족들은 국내에 들어와 독립운동을 펼쳤으며, 이들은 일본의 침략으로부터 나라를 지켜내는데 큰 공헌을 하였다.
이들에 맞서 대한민국 임시정부는 대한민국의 독립을 위해 헌신했고 그 공로를 인정받아 건국훈장 애족장을 추서하였다.
국가보훈처 공훈록에 따르면, 이들의 공적을 기려 훈장 1명, 대통령 표창 4명, 국무총리표


- 기존 문장에 있는것에 대해 하고 싶다면... BERT모델로. 새로 생성되는게 싫다면.. (헤드정보가 있어야함)

- BERT는 encode
- gpt는 decode
- Kobart는 encode, decode 둘다 가지고 있음